In [1]:
import random

from distributed_processing.serializers import JsonSerializer
from distributed_processing.client import Client
from distributed_processing.redis_connector import RedisConnector


In [2]:
REDIS_HOST = "localhost"
REDIS_PORT = 6379
REDIS_DB = 0
NAMESPACE = "tasks"

In [3]:
redis_connector = RedisConnector(redis_host=REDIS_HOST, redis_port=REDIS_PORT, 
                                 redis_db=REDIS_DB, namespace=NAMESPACE)

client = Client(JsonSerializer(), redis_connector, check_registry="cache")

Client with id: tasks:redis_client:2
Results queue: tasks:redis_client:2:responses


In [4]:
def f(x,y): return x + y

y = client.rpc_async_fn(f, [1, 2.0])

In [5]:
y.get()

3.0

In [6]:
fs =[]
tp = []
N = 10
for i in range(N):
    fn = random.choice(("add", "mul", "div"))
    t = (fn, [random.random(),random.random()], {})
    print(t)
    tp.append(t)
    fs.append(client.rpc_async(t[0], t[1]))

('add', [0.459801423003222, 0.26103117284264166], {})
('add', [0.8754862299982162, 0.6507492968317531], {})
('add', [0.11541865723176481, 0.6766062318812679], {})
('mul', [0.8214069437278143, 0.8868501079222512], {})
('div', [0.5410363339369209, 0.9771691085609299], {})
('div', [0.48020267066028877, 0.18991295799723806], {})
('mul', [0.6920001981266769, 0.5440263707268904], {})
('mul', [0.2683459042330425, 0.2981069816722991], {})
('mul', [0.995856210743214, 0.738673108866811], {})
('mul', [0.49120039595419074, 0.12766529214358313], {})


In [7]:
[f.get() for f in fs]

[0.7208325958458637,
 1.5262355268299692,
 0.7920248891130327,
 0.7284648366930986,
 0.5536772798044152,
 2.5285408416800736,
 0.37646635632914516,
 0.07999578755503613,
 0.735612203174012,
 0.06270924205053548]

In [8]:
fs = client.rpc_batch_async(tp)

In [9]:
[f.get() for f in fs]

[0.7208325958458637,
 1.5262355268299692,
 0.7920248891130327,
 0.7284648366930986,
 0.5536772798044152,
 2.5285408416800736,
 0.37646635632914516,
 0.07999578755503613,
 0.735612203174012,
 0.06270924205053548]

In [10]:
client.rpc_batch_sync(tp)

[0.7208325958458637,
 1.5262355268299692,
 0.7920248891130327,
 0.7284648366930986,
 0.5536772798044152,
 2.5285408416800736,
 0.37646635632914516,
 0.07999578755503613,
 0.735612203174012,
 0.06270924205053548]

In [11]:
fs =[]
tp = []
N = 10
for i in range(N):
    fn = random.choice(("add", "mul", "div", "kkk"))
    t = (fn, [random.random(),random.random()], {})
    print(t)
    tp.append(t)

('mul', [0.36478814351030686, 0.2742694285804175], {})
('add', [0.892620375508064, 0.7442188641687566], {})
('kkk', [0.6039452583969993, 0.6551412536337066], {})
('div', [0.7028342986861315, 0.7130466087388672], {})
('kkk', [0.8647336243080594, 0.30321518533595115], {})
('kkk', [0.6501366613278906, 0.2446289803523467], {})
('mul', [0.2741543596593893, 0.5048417500424516], {})
('mul', [0.405946292713147, 0.489719356247719], {})
('add', [0.1055945401902969, 0.12389914311025774], {})
('div', [0.7215552002600181, 0.3461803800004196], {})


In [12]:
try:
    fs = [client.rpc_async(t[0], t[1]) for t in tp]
except Exception as e:
    print(e)

Method kkk does not exist/is not available.


In [13]:
try:    
    fs = client.rpc_batch_async(tp)
except Exception as e:
    print(e)

Method kkk does not exist/is not available.


In [14]:
[f.safe_get(.1) for f in fs]

[]

In [15]:
try:
    x = client.rpc_batch_sync(tp, timeout=5)
except Exception as e:
    print(e)

Method kkk does not exist/is not available.


In [16]:
try:    
    x = client.rpc_batch_sync(tp, timeout=5)
except Exception as e:
    print(e)

Method kkk does not exist/is not available.


In [17]:
x = client.rpc_async("div", [1, 0])

In [18]:
try:
    x.get()
except Exception as e:
    print(e)

Error -32603 : Internal RPC error.

 Traceback (most recent call last):
  File "/home/augusto/tfm/distributed_processing/python/distributed_processing/worker.py", line 145, in _process_single_request
    result = func[request["method"]](*args, **kwargs)
  File "<ipython-input-13-cc39e5df0f9b>", line 13, in div
    return x / y
ZeroDivisionError: division by zero



In [19]:
client.check_registry="never"
client.set_default_queue("cola_1")

x = client.rpc_async("kk", [1, 0])

In [20]:
try:
    x.get()
except Exception as e:
    print(e)

Error -32601 : The method does not exist/is not available.

 


In [21]:
y = client.rpc_async("add", [1, 0])

In [22]:
y.get(5)

1

In [23]:
def f(x,y): return x + y

client.check_registry="never"
y = client.rpc_async_fn(f, [1, 2.0])
try:
    y.get()
except Exception as e:
    print(e)

Error -32601 : The method does not exist/is not available.

 
